In [21]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler

import implicit

In [22]:
raw_data = pd.read_csv('lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv',sep='\t',header=None)

In [23]:
raw_data = raw_data.drop(raw_data.columns[1], axis=1)

In [24]:
raw_data.columns = ['user','artist','play']

In [25]:
raw_data.head()

user                artist  play
0  00000c289a1829a808ac09c00daf10bc3c4e223b       betty blowtorch  2137
1  00000c289a1829a808ac09c00daf10bc3c4e223b             die Ärzte  1099
2  00000c289a1829a808ac09c00daf10bc3c4e223b     melissa etheridge   897
3  00000c289a1829a808ac09c00daf10bc3c4e223b             elvenking   717
4  00000c289a1829a808ac09c00daf10bc3c4e223b  juliette & the licks   706

In [26]:
data = raw_data.dropna()

In [27]:
data = data.copy()

In [28]:
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")

In [29]:
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

In [30]:
data['artist_id'].head()

0     45561
1     90933
2    185367
3    106704
4    155241
Name: artist_id, dtype: int32

In [31]:
sparse_item_user = sparse.csr_matrix((data['play'].astype(float), (data['artist_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['play'].astype(float), (data['user_id'], data['artist_id'])))

In [32]:
model = implicit.als.AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20)

In [33]:
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

In [34]:
model.fit(data_conf)

100%|██████████| 20.0/20 [02:18<00:00,  7.20s/it]


In [35]:
item_id = 147068 #Jay-Z
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print(data.artist.loc[data.artist_id == idx].iloc[0])

jay-z
snoop dogg
outkast
2pac
lupe fiasco
the roots
the streets
50 cent
rjd2
atmosphere


In [36]:
data.head()

user                artist  play  \
0  00000c289a1829a808ac09c00daf10bc3c4e223b       betty blowtorch  2137   
1  00000c289a1829a808ac09c00daf10bc3c4e223b             die Ärzte  1099   
2  00000c289a1829a808ac09c00daf10bc3c4e223b     melissa etheridge   897   
3  00000c289a1829a808ac09c00daf10bc3c4e223b             elvenking   717   
4  00000c289a1829a808ac09c00daf10bc3c4e223b  juliette & the licks   706   

   user_id  artist_id  
0        0      45561  
1        0      90933  
2        0     185367  
3        0     106704  
4        0     155241

In [37]:
# Create recommendations for user with id 2025
user_id = 0

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

print(recommendations)

                     artist     score
0          stephen sondheim  1.400877
1                  voltaire  1.381919
2                devil doll  1.317129
3                the donnas  1.261476
4              sweeney todd  1.259910
5    juliette and the licks  1.248622
6           letters to cleo  1.245984
7                   frente!  1.199383
8  zombina & the skeletones  1.183676
9                    yiruma  1.174877


In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark import SparkContext

In [2]:
spark_df = spark.read.csv('lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv',sep='\t')
spark_df.cache()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string]

In [3]:
spark_df.show()

+--------------------+--------------------+--------------------+----+
|                 _c0|                 _c1|                 _c2| _c3|
+--------------------+--------------------+--------------------+----+
|00000c289a1829a80...|3bd73256-3905-4f3...|     betty blowtorch|2137|
|00000c289a1829a80...|f2fb0ff0-5679-42e...|           die Ärzte|1099|
|00000c289a1829a80...|b3ae82c2-e60b-455...|   melissa etheridge| 897|
|00000c289a1829a80...|3d6bbeb7-f90e-4d1...|           elvenking| 717|
|00000c289a1829a80...|bbd2ffd7-17f4-450...|juliette & the licks| 706|
|00000c289a1829a80...|8bfac288-ccc5-448...|red hot chili pep...| 691|
|00000c289a1829a80...|6531c8b1-76ea-414...|              magica| 545|
|00000c289a1829a80...|21f3573f-10cf-44b...|the black dahlia ...| 507|
|00000c289a1829a80...|c5db90c4-580d-4f3...|         the murmurs| 424|
|00000c289a1829a80...|0639533a-0402-40b...|          lunachicks| 403|
|00000c289a1829a80...|a342964d-ca53-4e5...|    walls of jericho| 393|
|00000c289a1829a80..

In [4]:
spark_df = spark_df.drop(col("_c1"))

In [5]:
spark_df.count()

17559530

In [6]:
spark_df = spark_df.dropna(how='any')

In [57]:
spark_df.count()

17559516

In [7]:
from pyspark.sql.types import IntegerType
spark_df = spark_df.withColumn("_c3",spark_df["_c3"].cast(IntegerType()))

In [8]:
spark_df = spark_df.withColumnRenamed("_c0","user") \
                   .withColumnRenamed("_c2","artist") \
                   .withColumnRenamed("_c3","play")

In [9]:
spark_df.printSchema()

root
 |-- user: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- play: integer (nullable = true)



In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,VectorAssembler

In [11]:
indexer = [StringIndexer(inputCol=column,outputCol=column+"_index").fit(spark_df) for column in ["user","artist"]]
pipeline = Pipeline(stages=indexer)
spark_df = pipeline.fit(spark_df).transform(spark_df)
spark_df.cache()

DataFrame[user: string, artist: string, play: int, user_index: double, artist_index: double]

In [63]:
spark_df.show()

+--------------------+--------------------+----+----------+------------+
|                user|              artist|play|user_index|artist_index|
+--------------------+--------------------+----+----------+------------+
|00000c289a1829a80...|     betty blowtorch|2137|  145289.0|     17303.0|
|00000c289a1829a80...|           die Ärzte|1099|  145289.0|       277.0|
|00000c289a1829a80...|   melissa etheridge| 897|  145289.0|      3090.0|
|00000c289a1829a80...|           elvenking| 717|  145289.0|      1958.0|
|00000c289a1829a80...|juliette & the licks| 706|  145289.0|      5278.0|
|00000c289a1829a80...|red hot chili pep...| 691|  145289.0|         3.0|
|00000c289a1829a80...|              magica| 545|  145289.0|     10348.0|
|00000c289a1829a80...|the black dahlia ...| 507|  145289.0|       934.0|
|00000c289a1829a80...|         the murmurs| 424|  145289.0|      9298.0|
|00000c289a1829a80...|          lunachicks| 403|  145289.0|      8121.0|
|00000c289a1829a80...|    walls of jericho| 393|  1

In [12]:
spark_df.select([count(when(col(c).isNull(),c)).alias(c) for c in spark_df.columns]).show()

+----+------+----+----------+------------+
|user|artist|play|user_index|artist_index|
+----+------+----+----------+------------+
|   0|     0|   1|         0|           0|
+----+------+----+----------+------------+



In [13]:
spark_df = spark_df.dropna(how='any')
spark_df.cache()

DataFrame[user: string, artist: string, play: int, user_index: double, artist_index: double]

In [40]:
als = ALS(maxIter=5, regParam=0.01, userCol="user_index", itemCol="artist_index", ratingCol="play",
          coldStartStrategy="drop")
model = als.fit(spark_df)

In [71]:
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
# movieRecs = model.recommendForAllItems(10)

In [74]:
spark_df.show(2)

+--------------------+---------------+----+----------+------------+
|                user|         artist|play|user_index|artist_index|
+--------------------+---------------+----+----------+------------+
|00000c289a1829a80...|betty blowtorch|2137|  145289.0|     17303.0|
|00000c289a1829a80...|      die Ärzte|1099|  145289.0|       277.0|
+--------------------+---------------+----+----------+------------+
only showing top 2 rows



In [16]:
p = model.recommendForUserSubset(numItems=2,dataset=spark_df.filter(spark_df['user_index'] == 145289.0))

In [43]:
p.showw(2)

+----------+--------------------+
|user_index|     recommendations|
+----------+--------------------+
|    145289|[[264655, 115028....|
+----------+--------------------+



In [19]:
spark_df.filter(spark_df.user_index == 264655).show(10)

+--------------------+--------------------+----+----------+------------+
|                user|              artist|play|user_index|artist_index|
+--------------------+--------------------+----+----------+------------+
|263117187199a7a36...|   the acacia strain|1311|  264655.0|      3123.0|
|263117187199a7a36...|  the avett brothers| 950|  264655.0|      1975.0|
|263117187199a7a36...|        mewithoutyou| 673|  264655.0|      1211.0|
|263117187199a7a36...|           sigur rós| 626|  264655.0|        26.0|
|263117187199a7a36...|explosions in the...| 525|  264655.0|       199.0|
|263117187199a7a36...|        two gallants| 418|  264655.0|      3113.0|
|263117187199a7a36...|see you next tuesday| 337|  264655.0|      3641.0|
|263117187199a7a36...|this will destroy...| 213|  264655.0|      1521.0|
|263117187199a7a36...|            the bled| 182|  264655.0|      3179.0|
|263117187199a7a36...|      ion dissonance| 178|  264655.0|      4320.0|
+--------------------+--------------------+----+---

In [20]:
spark_df.filter(spark_df.user_index == 145289).show(10)

+--------------------+--------------------+----+----------+------------+
|                user|              artist|play|user_index|artist_index|
+--------------------+--------------------+----+----------+------------+
|00000c289a1829a80...|     betty blowtorch|2137|  145289.0|     17303.0|
|00000c289a1829a80...|           die Ärzte|1099|  145289.0|       277.0|
|00000c289a1829a80...|   melissa etheridge| 897|  145289.0|      3090.0|
|00000c289a1829a80...|           elvenking| 717|  145289.0|      1958.0|
|00000c289a1829a80...|juliette & the licks| 706|  145289.0|      5278.0|
|00000c289a1829a80...|red hot chili pep...| 691|  145289.0|         3.0|
|00000c289a1829a80...|              magica| 545|  145289.0|     10348.0|
|00000c289a1829a80...|the black dahlia ...| 507|  145289.0|       934.0|
|00000c289a1829a80...|         the murmurs| 424|  145289.0|      9298.0|
|00000c289a1829a80...|          lunachicks| 403|  145289.0|      8121.0|
+--------------------+--------------------+----+---

In [42]:
model.recommendForItemSubset(numUsers=2,dataset=spark_df.filter(spark_df['artist_index'] == 277.0)).show()

+------------+--------------------+
|artist_index|     recommendations|
+------------+--------------------+
|         277|[[157769, 235910....|
+------------+--------------------+

